In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import glob
import gc
# Date range 2018-08-13 to 2018-11-03
os.getcwd()


# Add the clicks

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Email_Analysis'

In [2]:
pd.options.display.max_columns = 999

In [3]:
df_all_DCM_LR_email_only_click=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Email_Analysis/dcm_lr_logs/df_click_file_all_matched_email_only_dict_JL_2019-07-23.csv",
                          dtype=str)

df_all_DCM_LR_email_only_impr=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Email_Analysis/dcm_lr_logs/df_impr_file_all_matched_email_only_dict_JL_2019-07-23.csv",
                          dtype=str)

print(df_all_DCM_LR_email_only_click.shape) # No email caught in the click files
print(df_all_DCM_LR_email_only_impr.shape)
df_all_DCM_LR_email_only_click.head(2)

(0, 9)
(15048035, 9)


,Customer_Link,Event Time,Advertiser ID,Campaign ID,Ad ID,Rendering ID,Creative Version,Site ID (DCM),Placement ID


In [4]:
placement_name=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Email_Analysis/mapping/placements_mapping_JL_20180813_201811032019-07-23.csv",
                           dtype=str,usecols=['Placement ID','Placement']).drop_duplicates()
site_name=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Email_Analysis/mapping/sites_mapping_JL_20180813_201811032019-07-23.csv",
                      dtype=str,usecols=['Site ID (DCM)','Site (DCM)']).drop_duplicates()

In [5]:
print(df_all_DCM_LR_email_only_impr['Placement ID'].unique())

df_all_DCM_LR_email_only_impr=pd.merge(df_all_DCM_LR_email_only_impr,site_name,on="Site ID (DCM)",how="left")
df_all_DCM_LR_email_only_impr.groupby(['Site (DCM)'])['Customer_Link'].count().to_frame().reset_index()

['224271181' '224418810' '224418537']


,Site (DCM),Customer_Link
0,Big Lots News,15048035


In [6]:
IDLs_in_email=df_all_DCM_LR_email_only_impr['Customer_Link'].unique().tolist()
print(len(IDLs_in_email))
del df_all_DCM_LR_email_only_click
del df_all_DCM_LR_email_only_impr
gc.collect()

1456216


53

In [7]:
df_all_DCM_LR_click=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Email_Analysis/dcm_lr_logs/df_click_file_all_matched_dict_JL_2019-07-23.csv",
                          dtype=str)
df_all_DCM_LR_impr=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Email_Analysis/dcm_lr_logs/df_impr_file_all_matched_dict_JL_2019-07-23.csv",
                          dtype=str)

In [8]:
df_all_DCM_LR_click['datetime']=df_all_DCM_LR_click['Event Time'].astype(int)
df_all_DCM_LR_click['datetime']=pd.to_datetime(df_all_DCM_LR_click['datetime']*1000)
df_all_DCM_LR_click['date']=df_all_DCM_LR_click['datetime'].apply(lambda x: x.date())
df_all_DCM_LR_impr['datetime']=df_all_DCM_LR_impr['Event Time'].astype(int)
df_all_DCM_LR_impr['datetime']=pd.to_datetime(df_all_DCM_LR_impr['datetime']*1000)
df_all_DCM_LR_impr['date']=df_all_DCM_LR_impr['datetime'].apply(lambda x: x.date())

print(df_all_DCM_LR_click.shape)
print(df_all_DCM_LR_impr.shape)

df_all_DCM_LR_click=df_all_DCM_LR_click[(df_all_DCM_LR_click['date']>=datetime.date(2018,8,13)) & (df_all_DCM_LR_click['date']<=datetime.date(2018,11,3)) ]
df_all_DCM_LR_impr=df_all_DCM_LR_impr[(df_all_DCM_LR_impr['date']>=datetime.date(2018,8,13)) & (df_all_DCM_LR_impr['date']<=datetime.date(2018,11,3)) ]

print(df_all_DCM_LR_click.shape)
print(df_all_DCM_LR_impr.shape)

(1800305, 11)
(48232542, 11)
(1800052, 11)
(48228917, 11)


In [9]:
df_all_DCM_LR_impr['media_impr']=np.where(~df_all_DCM_LR_impr['Placement ID'].isin(['224418810', '224418537', '224271181']),1,0)
df_all_DCM_LR_impr['email_impr']=np.where(df_all_DCM_LR_impr['Placement ID'].isin(['224418810', '224418537', '224271181']),1,0)

df_all_DCM_LR_click['media_click']=np.where(~df_all_DCM_LR_click['Placement ID'].isin(['224418810', '224418537', '224271181']),1,0)

df_impr_by_id_site=df_all_DCM_LR_impr.groupby(['Customer_Link','Site ID (DCM)'])['media_impr','email_impr'].sum().reset_index().rename(columns={"media_impr":"impressions"})
df_click_by_id_site=df_all_DCM_LR_click.groupby(['Customer_Link','Site ID (DCM)'])['media_click'].sum().to_frame().reset_index().rename(columns={"media_click":"click"})


In [10]:
df_dcm_by_id_site=pd.merge(df_impr_by_id_site,df_click_by_id_site,on=['Customer_Link','Site ID (DCM)'],how="outer")
df_dcm_by_id_site=df_dcm_by_id_site.fillna(0)

del df_impr_by_id_site
del df_click_by_id_site
gc.collect()

df_dcm_by_id_site.head(2)

,Customer_Link,Site ID (DCM),impressions,email_impr,click
0,XY1468---0ZPWrAcrcxY6_Oj-zOznn_mZ0wmpNMLTgWKNDxQQ,2479413,2.0,0.0,0.0
1,XY1468---9q1ArXNAg6NIU_2YMt-drlu04XlnVRHxKzKno6RE,2479413,2.0,0.0,0.0


In [11]:
try:
    os.stat("./output/")
except:
    os.mkdir("./output/")
    
df_dcm_by_id_site.to_csv("./output/df_dcm_by_id_site.csv",index=False)

In [12]:
'''
df_dcm_by_id_site_placement_others=df_dcm_by_id_site_placement[df_dcm_by_id_site_placement['email_3_placements']=="other_media"]
df_dcm_by_id_site_placement_emails=df_dcm_by_id_site_placement[df_dcm_by_id_site_placement['email_3_placements']!="other_media"]
del df_dcm_by_id_site_placement
gc.collect()
'''

'\ndf_dcm_by_id_site_placement_others=df_dcm_by_id_site_placement[df_dcm_by_id_site_placement[\'email_3_placements\']=="other_media"]\ndf_dcm_by_id_site_placement_emails=df_dcm_by_id_site_placement[df_dcm_by_id_site_placement[\'email_3_placements\']!="other_media"]\ndel df_dcm_by_id_site_placement\ngc.collect()\n'

In [13]:
df_dcm_by_id_site=pd.merge(df_dcm_by_id_site,site_name,on="Site ID (DCM)",how="left")
df_dcm_by_id_site=df_dcm_by_id_site.groupby(["Customer_Link","Site (DCM)"])['impressions','click','email_impr'].sum().reset_index()

df_dcm_by_id_site_wide_impr=df_dcm_by_id_site.pivot(index="Customer_Link",values="impressions",columns="Site (DCM)")
df_dcm_by_id_site_wide_click=df_dcm_by_id_site.pivot(index="Customer_Link",values="click",columns="Site (DCM)")
df_dcm_by_id_site_wide_email=df_dcm_by_id_site.pivot(index="Customer_Link",values="email_impr",columns="Site (DCM)")


df_dcm_by_id_site_wide_impr=df_dcm_by_id_site_wide_impr.reset_index()
df_dcm_by_id_site_wide_click=df_dcm_by_id_site_wide_click.reset_index()
df_dcm_by_id_site_wide_email=df_dcm_by_id_site_wide_email.reset_index()

df_dcm_by_id_site_wide_impr=df_dcm_by_id_site_wide_impr.fillna(0)
df_dcm_by_id_site_wide_click=df_dcm_by_id_site_wide_click.fillna(0)
df_dcm_by_id_site_wide_email=df_dcm_by_id_site_wide_email.fillna(0)

df_dcm_by_id_site_wide_impr.columns=['Customer_Link']+["impr_"+x for x in df_dcm_by_id_site_wide_impr.columns.tolist()[1:]]
df_dcm_by_id_site_wide_click.columns=['Customer_Link']+["click_"+x for x in df_dcm_by_id_site_wide_click.columns.tolist()[1:]]
df_dcm_by_id_site_wide_email.columns=['Customer_Link']+["email_"+x for x in df_dcm_by_id_site_wide_email.columns.tolist()[1:]]


In [14]:
df_by_id=pd.merge(df_dcm_by_id_site_wide_impr,df_dcm_by_id_site_wide_click,on="Customer_Link",how="outer")
df_by_id=pd.merge(df_by_id,df_dcm_by_id_site_wide_email,on="Customer_Link",how="outer")
for col in df_by_id.columns.tolist():
    if "impr_" in col:
        df_by_id[col]=df_by_id[col].fillna(0)
    elif "click_" in col:
        df_by_id[col]=df_by_id[col].fillna(0)
    elif "email_" in col:
        df_by_id[col]=df_by_id[col].fillna(0)
        
df_by_id['total_impr']=df_by_id[[x for x in df_by_id.columns.tolist() if "impr_" in x]].sum(axis=1)
df_by_id['total_click']=df_by_id[[x for x in df_by_id.columns.tolist() if "click_" in x]].sum(axis=1)
df_by_id['total_email']=df_by_id[[x for x in df_by_id.columns.tolist() if "email_" in x]].sum(axis=1)

df_by_id['impr_and_click']=df_by_id['total_impr']+df_by_id['total_click']


In [15]:
df_by_id['Expose_Group']=np.where(df_by_id['impr_and_click']==0,"email_only",
                                  np.where(df_by_id['impr_and_click']>=11,"click_impr_10+",
                                           np.where(df_by_id['impr_and_click']>=6,"click_impr_6-10","click_impr_1-5")
                                          )
                                 )

df_by_id['IDL_group']=np.where(df_by_id['Customer_Link'].isin(IDLs_in_email),"With_Email_idls","No_Email_idls")
df_by_id.head(2)

,Customer_Link,impr_Big Lots News,impr_Criteo,impr_DART Search : Google,impr_Facebook,"impr_Facebook Inc – Instagram, LLC",impr_Hulu.com,impr_Pandora,impr_Pinterest Inc.,impr_Spotify,impr_The Trade Desk,impr_Tremor Video,impr_YouTube.com,click_Big Lots News,click_Criteo,click_DART Search : Google,click_Facebook,"click_Facebook Inc – Instagram, LLC",click_Hulu.com,click_Pandora,click_Pinterest Inc.,click_Spotify,click_The Trade Desk,click_Tremor Video,click_YouTube.com,email_Big Lots News,email_Criteo,email_DART Search : Google,email_Facebook,"email_Facebook Inc – Instagram, LLC",email_Hulu.com,email_Pandora,email_Pinterest Inc.,email_Spotify,email_The Trade Desk,email_Tremor Video,email_YouTube.com,total_impr,total_click,total_email,impr_and_click,Expose_Group,IDL_group
0,XY1468---0ZPWrAcrcxY6_Oj-zOznn_mZ0wmpNMLTgWKNDxQQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,click_impr_1-5,No_Email_idls
1,XY1468---9q1ArXNAg6NIU_2YMt-drlu04XlnVRHxKzKno6RE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,click_impr_1-5,No_Email_idls


In [16]:
sales_file_list=glob.glob("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/*.txt")

sales_file_df=pd.DataFrame({"file_path":sales_file_list})
sales_file_df['week_end_dt']=sales_file_df['file_path'].apply(lambda x: x.split("MediaStormDailySalesHistory")[1][:10])
sales_file_df=sales_file_df[sales_file_df['week_end_dt']>="20180811"]
sales_file_df=sales_file_df[sales_file_df['week_end_dt']<="20181103"]


sales_file_list=sorted(sales_file_df['file_path'].tolist())
len(sales_file_list)

12

In [17]:
df_all_sales=pd.DataFrame()
i_counter=0
for file in sales_file_list:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','item_transaction_units','item_transaction_amt'])
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df=df[pd.notnull(df['customer_id_hashed'])]
    df['Trans_Group']=np.where(df['location_id']=="6990","Online","Instore")
    
    df_sales=df.groupby(["customer_id_hashed","Trans_Group",'transaction_dt'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"sales"})
    
    df_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed','Trans_Group']].drop_duplicates()
    df_trans=df_trans.groupby(["customer_id_hashed","Trans_Group",'transaction_dt'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans"})
    df=pd.merge(df_sales,df_trans,on=["customer_id_hashed","Trans_Group",'transaction_dt'],how="outer")
    
    df_all_sales=df_all_sales.append(df)
    i_counter+=1
    print(i_counter,datetime.datetime.now())
print(df_all_sales.shape)
del df
gc.collect()

1 2019-07-24 22:51:13.157511
2 2019-07-24 22:52:35.427123
3 2019-07-24 22:55:04.483744
4 2019-07-24 22:56:37.969496
5 2019-07-24 22:57:55.119099
6 2019-07-24 22:59:08.306695
7 2019-07-24 23:00:25.308926
8 2019-07-24 23:02:06.000756
9 2019-07-24 23:04:04.943804
10 2019-07-24 23:05:28.794659
11 2019-07-24 23:06:56.700664
12 2019-07-24 23:08:43.516653
(14760036, 5)


157

In [18]:
df_all_sales.to_csv("./output/df_all_sales_by_day_id.csv",index=False)

In [19]:
df_all_sales_overall=df_all_sales.groupby(['customer_id_hashed','Trans_Group'])['sales','trans'].sum().reset_index()
df_all_sales_overall_wide_sales=df_all_sales_overall.pivot(index="customer_id_hashed",values="sales",columns="Trans_Group")
df_all_sales_overall_wide_trans=df_all_sales_overall.pivot(index="customer_id_hashed",values="trans",columns="Trans_Group")


In [20]:
df_all_sales_overall_wide_sales=df_all_sales_overall_wide_sales.reset_index()
df_all_sales_overall_wide_trans=df_all_sales_overall_wide_trans.reset_index()

df_all_sales_overall_wide_sales=df_all_sales_overall_wide_sales.fillna(0)
df_all_sales_overall_wide_trans=df_all_sales_overall_wide_trans.fillna(0)

df_all_sales_overall_wide_sales.columns=['customer_id_hashed','sales_Instore','sales_Online']
df_all_sales_overall_wide_trans.columns=['customer_id_hashed','trans_Instore','trans_Online']

In [21]:
df_all_sales_overall_wide=pd.merge(df_all_sales_overall_wide_sales,df_all_sales_overall_wide_trans,on=["customer_id_hashed"],how="outer")
df_all_sales_overall_wide=df_all_sales_overall_wide.fillna(0)

In [22]:
df_all_sales_overall_wide.to_csv("./output/df_all_sales_overall_wide.csv",index=False)


In [23]:
mapping_1="./id_mapping/CL_BigLots_allrewards_combined0908_20181029_171317_0000.psv.gz"
mapping_2="./id_mapping/CL_BigLots_allrewards_combined1011_20181026_232208_0000.psv.gz"

df_mapping_1=pd.read_table(mapping_1,dtype=str,sep="|",compression="gzip")
df_mapping_2=pd.read_table(mapping_2,dtype=str,sep="|",compression="gzip")

df_mapping=df_mapping_2.append(df_mapping_1)
df_mapping=df_mapping[['Customer_Link','customer_id_hashed']].drop_duplicates()
print(df_mapping.shape)
print(df_mapping['Customer_Link'].nunique())
print(df_mapping['customer_id_hashed'].nunique())

del df_mapping_1
del df_mapping_2
df_mapping=df_mapping.drop_duplicates("Customer_Link")
gc.collect()
print(df_mapping.shape)

gc.collect()

(22292543, 2)
21984450
22278790
(21984450, 2)


0

In [24]:
df_by_id_with_sales=pd.merge(df_by_id,df_mapping,on="Customer_Link",how="left")
df_by_id_with_sales=pd.merge(df_by_id_with_sales,df_all_sales_overall_wide,on="customer_id_hashed",how="left")

In [25]:
df_by_id_with_sales.to_csv("./output/df_by_id_with_sales.csv",index=False)

In [26]:
df_by_id_with_sales['sales_Instore']=df_by_id_with_sales['sales_Instore'].fillna(0)
df_by_id_with_sales['sales_Online']=df_by_id_with_sales['sales_Online'].fillna(0)
df_by_id_with_sales['trans_Instore']=df_by_id_with_sales['trans_Instore'].fillna(0)
df_by_id_with_sales['trans_Online']=df_by_id_with_sales['trans_Online'].fillna(0)

df_by_id_with_sales['total_sales']=df_by_id_with_sales['sales_Instore']+df_by_id_with_sales['sales_Online']
df_by_id_with_sales['total_trans']=df_by_id_with_sales['trans_Instore']+df_by_id_with_sales['trans_Online']

df_by_id_with_sales['BL_shoppers']=np.where(df_by_id_with_sales['total_trans']==0,0,1)

In [27]:
df_by_id_with_sales['rewards_group']=np.where(pd.isnull(df_by_id_with_sales['customer_id_hashed']),"Non_Rewards","Rewards")


In [28]:
for col in df_by_id_with_sales.columns.tolist():
    print(col,df_by_id_with_sales[col].apply(lambda x: type(x)).unique())

Customer_Link [<class 'str'>]
impr_Big Lots News [<class 'float'>]
impr_Criteo [<class 'float'>]
impr_DART Search : Google [<class 'float'>]
impr_Facebook [<class 'float'>]
impr_Facebook Inc – Instagram, LLC [<class 'float'>]
impr_Hulu.com [<class 'float'>]
impr_Pandora [<class 'float'>]
impr_Pinterest Inc. [<class 'float'>]
impr_Spotify [<class 'float'>]
impr_The Trade Desk [<class 'float'>]
impr_Tremor Video [<class 'float'>]
impr_YouTube.com [<class 'float'>]
click_Big Lots News [<class 'float'>]
click_Criteo [<class 'float'>]
click_DART Search : Google [<class 'float'>]
click_Facebook [<class 'float'>]
click_Facebook Inc – Instagram, LLC [<class 'float'>]
click_Hulu.com [<class 'float'>]
click_Pandora [<class 'float'>]
click_Pinterest Inc. [<class 'float'>]
click_Spotify [<class 'float'>]
click_The Trade Desk [<class 'float'>]
click_Tremor Video [<class 'float'>]
click_YouTube.com [<class 'float'>]
email_Big Lots News [<class 'float'>]
email_Criteo [<class 'float'>]
email_DART Sear

In [29]:
df_output_group_shoppers_count=df_by_id_with_sales.groupby(["IDL_group","Expose_Group","rewards_group"])['customer_id_hashed','Customer_Link'].nunique().reset_index()
df_output_group_shoppers_count=df_output_group_shoppers_count.rename(columns={"customer_id_hashed":"BL_rewards_ids","Customer_Link":"LR_IDLs"})


In [30]:
col_list_sum=[x for x in df_by_id_with_sales.columns.tolist() if (x not in ['Customer_Link','Expose_Group','IDL_group','customer_id_hashed'])]
df_output_group_sum=df_by_id_with_sales.groupby(["IDL_group","Expose_Group","rewards_group"])[col_list_sum].sum().reset_index()


In [31]:
df_output_group=pd.merge(df_output_group_shoppers_count,df_output_group_sum,on=["IDL_group","Expose_Group","rewards_group"],how="outer")


In [32]:
df_output_group

,IDL_group,Expose_Group,rewards_group,BL_rewards_ids,LR_IDLs,impr_Big Lots News,impr_Criteo,impr_DART Search : Google,impr_Facebook,"impr_Facebook Inc – Instagram, LLC",impr_Hulu.com,impr_Pandora,impr_Pinterest Inc.,impr_Spotify,impr_The Trade Desk,impr_Tremor Video,impr_YouTube.com,click_Big Lots News,click_Criteo,click_DART Search : Google,click_Facebook,"click_Facebook Inc – Instagram, LLC",click_Hulu.com,click_Pandora,click_Pinterest Inc.,click_Spotify,click_The Trade Desk,click_Tremor Video,click_YouTube.com,email_Big Lots News,email_Criteo,email_DART Search : Google,email_Facebook,"email_Facebook Inc – Instagram, LLC",email_Hulu.com,email_Pandora,email_Pinterest Inc.,email_Spotify,email_The Trade Desk,email_Tremor Video,email_YouTube.com,total_impr,total_click,total_email,impr_and_click,sales_Instore,sales_Online,trans_Instore,trans_Online,total_sales,total_trans,BL_shoppers
0,No_Email_idls,click_impr_1-5,Non_Rewards,0,3835346,0.0,804303.0,0.0,0.0,0.0,9857.0,4243.0,0.0,137.0,4929529.0,1.0,0.0,0.0,3637.0,867062.0,72448.0,4348.0,21.0,39.0,0.0,245.0,25618.0,0.0,1686.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5748070.0,975104.0,0.0,6723174.0,0.000000e+00,0.00,0.0,0.0,0.000000e+00,0.0,0
1,No_Email_idls,click_impr_1-5,Rewards,777349,777355,0.0,180699.0,0.0,0.0,0.0,2249.0,778.0,0.0,38.0,1139536.0,0.0,0.0,0.0,1019.0,158139.0,40878.0,1431.0,3.0,13.0,0.0,70.0,6818.0,0.0,808.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1323300.0,209179.0,0.0,1532479.0,2.373925e+07,229887.53,628478.0,2833.0,2.396913e+07,631311.0,270754
2,No_Email_idls,click_impr_10+,Non_Rewards,0,364909,0.0,9628937.0,0.0,0.0,0.0,1671.0,2210.0,0.0,227.0,3276937.0,5.0,0.0,0.0,71249.0,156791.0,9117.0,82.0,1.0,3.0,0.0,4.0,11632.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12909987.0,248899.0,0.0,13158886.0,0.000000e+00,0.00,0.0,0.0,0.000000e+00,0.0,0
3,No_Email_idls,click_impr_10+,Rewards,139972,139972,0.0,3303426.0,0.0,0.0,0.0,623.0,834.0,0.0,126.0,1878541.0,0.0,0.0,0.0,23257.0,46415.0,9079.0,181.0,0.0,1.0,0.0,5.0,7840.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5183550.0,86798.0,0.0,5270348.0,6.801611e+06,189988.73,150842.0,2246.0,6.991600e+06,153088.0,58745
4,No_Email_idls,click_impr_6-9,Non_Rewards,0,346316,0.0,865358.0,0.0,0.0,0.0,1749.0,1103.0,0.0,65.0,1601063.0,6.0,0.0,0.0,5207.0,112109.0,8804.0,270.0,0.0,4.0,8.0,17.0,9943.0,1.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2469344.0,136423.0,0.0,2605767.0,0.000000e+00,0.00,0.0,0.0,0.000000e+00,0.0,0
5,No_Email_idls,click_impr_6-9,Rewards,117302,117302,0.0,213023.0,0.0,0.0,0.0,527.0,335.0,0.0,31.0,630779.0,0.0,0.0,0.0,1371.0,30220.0,7332.0,168.0,1.0,3.0,0.0,1.0,4553.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,844695.0,43673.0,0.0,888368.0,4.386242e+06,65330.59,105533.0,834.0,4.451573e+06,106367.0,43782
6,With_Email_idls,click_impr_1-5,Non_Rewards,0,83930,0.0,45183.0,0.0,0.0,0.0,118.0,84.0,0.0,3.0,106878.0,0.0,0.0,0.0,238.0,18440.0,1309.0,50.0,0.0,1.0,0.0,5.0,392.0,0.0,14.0,820613.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,152266.0,20449.0,820613.0,172715.0,0.000000e+00,0.00,0.0,0.0,0.000000e+00,0.0,0
7,With_Email_idls,click_impr_1-5,Rewards,73214,73216,0.0,29252.0,0.0,0.0,0.0,146.0,74.0,0.0,5.0,108764.0,0.0,0.0,0.0,128.0,12447.0,2554.0,96.0,0.0,0.0,0.0,3.0,417.0,0.0,11.0,836336.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,138241.0,15656.0,836336.0,153897.0,3.325026e+06,67810.77,92578.0,844.0,3.392837e+06,93422.0,34767
8,With_Email_idls,click_impr_10+,Non_Rewards,0,40539,0.0,2077240.0,0.0,0.0,0.0,63.0,80.0,0.0,0.0,249255.0,0.0,0.0,0.0,17511.0,11759.0,785.0,13.0,0.0,0.0,0.0,0.0,725.0,0.0,0.0,464618.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2326638.0,30793.0,464618.0,2357431.0,0.000000e+00,0.00,0.0,0.0,0.000000e+00,0.0,0
9,With_Email_idls,click_impr_10+,Rewards,33213,33213,0.0,1538018.0,0.0,0.0,0.0,21.0,75.0,0.0,14.0,305057.0,0.0,0.0,0.0,12755.0,8474.0,1435.0,5.0,0.0,0.0,0.0,1.0,866.0,0.0,3.0,439779.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1

In [33]:
df_output_group.to_csv("./output/BL_df_output_group_rewards_JL_"+str(datetime.datetime.now().date())+".csv",index=False)